In [1]:
#First import the dataset. Check sample size and number of features:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
df_orig=pd.read_csv("/Users/LJin/Documents/ds-case-study-linear-models-master/predict_auction_price/data/Train.csv")
df_orig.shape

/Users/LJin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(401125, 53)

In [2]:
#Look at types of columns
dtype_df = df_orig.dtypes.reset_index()
dtype_df.columns=['count','column type']
dtype_df.groupby('column type').aggregate('count')
df_column_null = df_orig.isnull().sum().sort_values(ascending=False).reset_index()

#Get rid of features that have more than 1/4 of total sample missing values
df_column_null.columns = ['feature','count_of_missing']
df_column_removed20miss = df_column_null[df_column_null['count_of_missing']<df_orig.shape[0]/4].reset_index()

#df is the new dataframe after removing those features
df_keep = df_orig[list(df_column_removed20miss['feature'])]

In [3]:
#Drop all the rows that have missing values. Updated dataframe is called df_keep
df_keep = df_keep.dropna(how='any')
df_keep.columns

Index(['Hydraulics', 'auctioneerID', 'Enclosure', 'ProductGroupDesc',
       'YearMade', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'saledate', 'ProductGroup', 'fiModelDesc', 'fiBaseModel',
       'fiProductClassDesc', 'state', 'SalesID'],
      dtype='object')

In [4]:
#Change 'YearMade' 1000 to missing values, then fill them with median year
df_keep['YearMade']=np.where(df_keep['YearMade']==1000,df_keep['YearMade'].median(),df_keep['YearMade'])

#Create new features 'Age' based on 'YearMade'
df_keep['Age'] = (datetime.datetime.now().year - df_keep['YearMade'])

#create new feature 'sale_year' based on 'saledate'
import datetime
df_keep['sale_year'] = df_keep['saledate'].apply(lambda x: pd.datetime.strptime(x, '%m/%d/%Y %H:%M').year)

In [5]:
#Standardize 'Age' and 'sale_year'
from sklearn.preprocessing import StandardScaler
Scaler=StandardScaler()
df_keep['sale_year']=Scaler.fit_transform(df_keep[['sale_year','Age']])[:,0]
df_keep['Age']=Scaler.fit_transform(df_keep[['sale_year','Age']])[:,1]

In [6]:
#do a log transformation on y 'SalePrice', call it 'SalePrice_log'
df_keep['SalePrice_log']=np.log(df_keep['SalePrice'])

In [7]:
#For the sake of simplicity, remove some more features that SEEM unrelated, such as 'Hydraulics','ProductGroupDesc'
#(a high correlation with 'ProductGroup'), 'MachineID', 'ModelID', 'fiModelDesc', 'fiBaseModel','SalesID'

#here's the final dataframe df
df=df_keep[['auctioneerID','Enclosure','ProductGroup','datasource','state','Age','sale_year','SalePrice_log']]
df.head()

,auctioneerID,Enclosure,ProductGroup,datasource,state,Age,sale_year,SalePrice_log
0,3.0,EROPS w AC,WL,121,Alabama,-1.112415,0.352413,11.097410
1,3.0,EROPS w AC,WL,121,North Carolina,-0.287068,0.007787,10.950807
2,3.0,OROPS,SSL,121,New York,-0.802910,0.007787,9.210340
3,3.0,EROPS w AC,TEX,121,Texas,-0.802910,1.213975,10.558414
4,3.0,EROPS,SSL,121,New York,-1.421919,0.869350,9.305651


In [8]:
#transform categorical to dummy variables
df['auctioneerID']=df['auctioneerID'].astype(str)
df['datasource']=df['datasource'].astype(str)
df=pd.get_dummies(df,columns=['auctioneerID','Enclosure','ProductGroup','state','datasource'])

/Users/LJin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/LJin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
#separate the dataframe to X and y, prepare for model fitting
X=df.drop('SalePrice_log',1)
y=df['SalePrice_log']

#split X and y into train and test
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape,X_test.shape

((244120, 98), (61031, 98))

In [10]:
#create linear model called 'estimator', use RFECV on 'estimator', call it 'Selector'
from sklearn.feature_selection import RFECV
from sklearn import linear_model
estimator = linear_model.LinearRegression()
Selector = RFECV(estimator)

In [ ]:
#trying to fit 'Selector' gives me the problem......
Selector.fit(X_train,y_train)